In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

#loading datasets
try:
    df_sales =pd.read_csv('data.csv')
    df_targets = pd.read_csv('output.csv')
    print("Files loaded successfully")
except FileNotFoundError:
    print("Error. Check file names.")

#displaying first 5 rows
df_sales.head()

Files loaded successfully


,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [2]:
# EDA and CLEANING
print("---- 1. Basic info ----")
df_sales.info() #tells us data types, and non-null counts

print("\n ---- 2. Missing values amount ----")
print(df_sales.isnull().sum()) #missing value amount

print("\n ---- 3. Numerical data summary ----")
print(df_sales.describe()) # number column statistics, and weird values

print("\n ---- 4. Categorical data summary ----")
print(df_sales.describe(include = 'object')) #text column statistics; unique = how many different regions there are, and top = most common one

---- 1. Basic info ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   int64  
 8   view           4600 non-null   int64  
 9   condition      4600 non-null   int64  
 10  sqft_above     4600 non-null   int64  
 11  sqft_basement  4600 non-null   int64  
 12  yr_built       4600 non-null   int64  
 13  yr_renovated   4600 non-null   int64  
 14  street         4600 non-null   object 
 15  city           4600 non-null   object 
 16  statezip       4600 non-null   object 
 17  country        4600 non-null

In [6]:
# CLEANING the data

df_clean = df_sales.copy() #Create copy to keep original data safe

#There's a house price of 0; we need to remove it. So, here we remove rows where the price is equal to 0
start_rows = len(df_clean) #n.o. rows at the beginning
df_clean = df_clean[df_clean['price'] > 0] #remove prices less than 0; they make no sense.
end_rows = len(df_clean) #n.o. rows left after cleaning
print(f"Removed {start_rows - end_rows} rows with $0 price.")

#We've found a useless column. Country column has only 1 unique value (USA) - it won't give any info for analysis, so we drop it.
df_clean = df_clean.drop(columns=['country'])

#Date is an objec, but we need to change it to a datetime object to allow us to do a time-based analysis
df_clean['date'] = pd.to_datetime(df_clean['date'])

print("Data cleaning complete :)")

Removed 49 rows with $0 price.
Data cleaning complete :)


In [8]:
# ANALYSIS for DASHBOARD

print("\n ---- Median price by city ----")
median_price_by_city = df_clean.groupby('city')['price'].median().sort_values(ascending=False) #the mean would be affected by a couple mansons so the median was taken here as the better fit
print(median_price_by_city.head(10)) # showing the 10 most expensive cities

print("\n ---- Price by living area ----")
sqft_vs_price = df_clean[['sqft_living', 'price']] #relationship between sqft_living and price
print(sqft_vs_price.head()) #showing what it looks like

print("\n Price by overall quality")
median_price_by_condition = df_clean.groupby('condition')['price'].median().sort_values(ascending=False) #gives the median price for each condition level. The condition has quality (1-5).
print(median_price_by_condition)

#exporting for Tableau
df_clean.to_csv('house_data_cleaned.csv', index=False) #saving the cleaned data to a CSV file
print("\n Success :) 'house_data_cleaned.csv' is available for Tableau.")


 ---- Median price by city ----
city
Medina                2.100000e+06
Yarrow Point          1.397000e+06
Clyde Hill            1.325000e+06
Mercer Island         9.504667e+05
Beaux Arts Village    7.450000e+05
Bellevue              7.250000e+05
Sammamish             6.685000e+05
Redmond               6.380000e+05
Newcastle             6.025000e+05
Fall City             5.635000e+05
Name: price, dtype: float64

 ---- Price by living area ----
   sqft_living      price
0         1340   313000.0
1         3650  2384000.0
2         1930   342000.0
3         2000   420000.0
4         1940   550000.0

 Price by overall quality
condition
5    558000.0
3    465950.0
4    440000.0
1    310000.0
2    250000.0
Name: price, dtype: float64

 Success :) 'house_data_cleaned.csv' is available for Tableau.


In [9]:
# --- Regression Model ---

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

print("--- Starting on: Predictive Model ---")

# 1. Define Features (X) and Target (y)
# We need to select the "clues" our model will use.
# 'df_clean' is the DataFrame we created in the previous cell.

# Let's select our features (the "clues")
# We'll pick a mix of number and text columns
X = df_clean[['sqft_living', 'bedrooms', 'bathrooms', 'floors', 'condition', 'city']]

# This is our target (the "answer" we want to predict)
y = df_clean['price']


# 2. Handle the "Skew" in Price 
# Prices are heavily skewed. Taking the log makes the model much more accurate.
# We'll use np.log1p() which also handles any 0s well.
y_log = np.log1p(y)

print("Target variable 'price' has been log-transformed.")

# 3. Define Preprocessing Steps
# Need to convert text ('city') to numbers and scale our numeric data.

# Get lists of column names
numeric_features = ['sqft_living', 'bedrooms', 'bathrooms', 'floors', 'condition']
categorical_features = ['city']

# Create the "preprocessor"
# OneHotEncoder handles text, StandardScaler handles numbers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 4. Create the Model Pipeline
# This bundles our preprocessing and our model into one step.
# We are using 'LinearRegression' this time, not a classifier.
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', LinearRegression())
                                ])

# 5. Split Data into Training and Testing
# We train on 80% of the data, and test on the unseen 20%
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

print("Data split. Training model...")

# 6. Train the Model
model_pipeline.fit(X_train, y_train_log)

print("Model training complete!")

# 7. Evaluate the Model
# First, get the predictions
y_pred_log = model_pipeline.predict(X_test)

# IMPORTANT: We have to convert our predictions back from log scale
# We use np.expm1() which is the reverse of np.log1p()
y_pred = np.expm1(y_pred_log)
y_test = np.expm1(y_test_log) # Also convert our test 'answers' back

# Now we can calculate our metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("\n--- Model Evaluation Report ---")
print(f"R-squared (R²): {r2:.3f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:,.2f}")
print("\n---")
print(f"R-squared (R²): {r2:.3f} --- This means our model can explain {r2:.1%} of the variation in house prices.")
print(f"RMSE: ${rmse:,.2f} --- This means, on average, our model's price prediction is off by about ${rmse:,.2f}.")

--- Starting on: Predictive Model ---
Target variable 'price' has been log-transformed.
Data split. Training model...
Model training complete!

--- Model Evaluation Report ---
R-squared (R²): -6.677
Root Mean Squared Error (RMSE): $1,068,682.54

---
R-squared (R²): -6.677 --- This means our model can explain -667.7% of the variation in house prices.
RMSE: $1,068,682.54 --- This means, on average, our model's price prediction is off by about $1,068,682.54.


In [10]:
#Linear regression model is a bad one for this data. So, we need to use something else.
#The R^2 value is a negative value and this means the model is much worse than just guessing the house prices.

# --- Using a Better Model ---

# We're importing RandomForestRegressor instead of LinearRegression
from sklearn.ensemble import RandomForestRegressor

print("--- Starting: RandomForestRegressor ---")

# The setup is all the same
# X, y, y_log, preprocessor, X_train, X_test...

# 4. Create the NEW Model Pipeline
# Swapping 'LinearRegression()' to 'RandomForestRegressor()'
model_pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', RandomForestRegressor(random_state=42)) 
                                ]) # n_jobs=-1 makes it train faster

print("Data split. Training NEW model...")

# 6. Train the NEW Model
# This will take longer, as it's a much bigger model.
model_pipeline_rf.fit(X_train, y_train_log)

print("Model training complete!")

# 7. Evaluate the NEW Model
y_pred_log_rf = model_pipeline_rf.predict(X_test)

# Convert back from log scale
y_pred_rf = np.expm1(y_pred_log_rf)
y_test_rf = np.expm1(y_test_log) # Using the same test data as before

# Calculate our new metrics
r2_rf = r2_score(y_test_rf, y_pred_rf)
mse_rf = mean_squared_error(y_test_rf, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print("\n--- NEW Model Evaluation Report (RandomForest) ---")
print(f"R-squared (R²): {r2_rf:.3f}")
print(f"Root Mean Squared Error (RMSE): ${rmse_rf:,.2f}")
print("\n---")
print(f"R-squared (R²): {r2_rf:.3f} --- This means our model can explain {r2_rf:.1%} of the variation in house prices.")
print(f"RMSE: ${rmse_rf:,.2f} --- This means, on average, our model's price prediction is off by about ${rmse_rf:,.2f}.")

--- Starting: RandomForestRegressor ---
Data split. Training NEW model...
Model training complete!

--- NEW Model Evaluation Report (RandomForest) ---
R-squared (R²): 0.619
Root Mean Squared Error (RMSE): $238,104.10

---
R-squared (R²): 0.619 --- This means our model can explain 61.9% of the variation in house prices.
RMSE: $238,104.10 --- This means, on average, our model's price prediction is off by about $238,104.10.
